In [1]:
%pip install -r requirements.txt

/bin/bash: /home/henriquesabino/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym    
import slimevolleygym
from slimevolleygym import SurvivalRewardEnv
import numpy as np
from gym.wrappers.gray_scale_observation import GrayScaleObservation
from gym.wrappers.resize_observation import ResizeObservation
from atari_wrappers import RenderWrapper, BufferWrapper, ImageToPyTorch

INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SlimeVolleyNoFrameskip-v0'
env = gym.make(env_name)
env = SurvivalRewardEnv(env)
env = RenderWrapper(env)
env = ResizeObservation(env, INPUT_SHAPE)
env = GrayScaleObservation(env, True)
env = ImageToPyTorch(env)
env = BufferWrapper(env, WINDOW_LENGTH, np.uint8)

/home/henriquesabino/miniconda3/lib/python3.11/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/henriquesabino/miniconda3/lib/python3.11/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback, EveryNTimesteps
from train_metrics import ModelMetricsCallback
from model_utils import createOrLoadModel, recordVideoForModel
    
model, steps_done, mean_rewards, rewards_stds = createOrLoadModel('dqn', env, 'dqn_', '_steps.zip', 'dqn')

eval_env = gym.make(env_name)
eval_env = ResizeObservation(eval_env, INPUT_SHAPE)
eval_env = GrayScaleObservation(eval_env, True)
eval_env = ImageToPyTorch(eval_env)
eval_env = BufferWrapper(eval_env, WINDOW_LENGTH, np.uint8)

checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='./models', name_prefix='dqn')
model_metrics_callback = ModelMetricsCallback(eval_env, './models', num_episodes=30, mean_rewards=mean_rewards, rewards_stds=rewards_stds, verbose=0)
metrics_callback = EveryNTimesteps(n_steps=1000, callback=model_metrics_callback)

callbacks=[checkpoint_callback, metrics_callback]
model = model.learn(total_timesteps=(1e6 - steps_done), log_interval=4, callback=callbacks)

recordVideoForModel('dqn')

In [ ]:
from plotting import plot_result

mean_rewards = model_metrics_callback.mean_rewards

plot_result(mean_rewards, -4, 1000)